In [1]:
from spikeinterface import create_sorting_analyzer
from spikeinterface import create_sorting_analyzer
import spikeinterface.widgets as sw
from spikeinterface.curation import compute_merge_unit_groups

from pathlib import Path

import spikeinterface as si
import numpy as np

# Correct import path
from spikeinterface.generation import generate_drifting_recording

# Generate synthetic drifting recording
recording, rec_drift, sorting = generate_drifting_recording(duration=120)

analyzer = create_sorting_analyzer(sorting=sorting, recording=recording)

# some extensions are required
analyzer.compute(["random_spikes", "templates", "template_similarity", "correlograms", "noise_levels"])
analyzer.compute("unit_locations", method="monopolar_triangulation")
analyzer.compute("random_spikes", method="uniform", max_spikes_per_unit=500)
analyzer.compute("waveforms", n_jobs=-1, chunk_duration="1s")
analyzer.compute("templates")
analyzer.compute("correlograms", window_ms=100.0, bin_ms=1.0)
analyzer.compute("spike_amplitudes")
analyzer.compute("unit_locations")

/opt/anaconda3/envs/sorter/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
estimate_sparsity (no parallelization): 100%|██████████| 120/120 [00:11<00:00, 10.73it/s]
estimate_templates_with_accumulator (no parallelization): 100%|██████████| 120/120 [00:18<00:00,  6.55it/s]
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
noise_level (no parallelization): 100%|██████████| 20/20 [00:03<00:00,  6.54it/s]


Deleting templates
Deleting template_similarity
Deleting unit_locations


spike_amplitudes (no parallelization): 100%|██████████| 120/120 [00:20<00:00,  5.87it/s]


In [ ]:
fig_dir = Path(r"/Users/marco/Cursor_Folder/Cursor_Codespace/realtime_sorting/data/figs")
fig_dir.mkdir(parents = True, exist_ok = True)

for id in analyzer.unit_ids:
    w = sw.plot_unit_summary(
    analyzer,
    unit_id=id,
    backend="matplotlib",
    ncols=5,
    figsize=(16, 9),
    figtitle=f"Unit summary: {id}"
    )

    w.figure.savefig(
        Path(fig_dir, f"{id}-figure")
    )

In [2]:
from spikeinterface.curation import auto_merge_units

template_diff_thresh = [0.05, 0.15, 0.25]
presets = ["x_contaminations"] * len(template_diff_thresh)
steps_params = [
    {"template_similarity": {"template_diff_thresh": i}}
    for i in template_diff_thresh
]

analyzer_merged = auto_merge_units(
    analyzer,
    presets=presets,
    steps_params=steps_params,
    recursive=True
)

In [4]:
analyzer.save_as(format = "zarr", folder = "/Users/marco/Cursor_Folder/Cursor_Codespace/realtime_sorting/data/analyzer")

SortingAnalyzer: 128 channels - 250 units - 1 segments - zarr - sparse - has recording
Loaded 6 extensions: random_spikes, templates, template_similarity, correlograms, noise_levels, unit_locations

In [ ]:
!sigui --mode=web --curation "/Users/marco/Cursor_Folder/Cursor_Codespace/realtime_sorting/data/analyzer.zarr"

Found available port: 54172
Launching server at http://localhost:54172


In [ ]:
from pathlib import Path
fig_dir = Path(r"/Users/marco/Cursor_Folder/Cursor_Codespace/realtime_sorting/data/figs")
fig_dir.mkdir(parents = True, exist_ok = True)

for id in analyzer.unit_ids:
    w = sw.plot_unit_summary(
    analyzer,
    unit_id=id,
    backend="matplotlib",
    ncols=5,
    figsize=(16, 9),
    figtitle=f"Unit summary: {id}"
    )

    w.figure.savefig(
        Path(fig_dir, f"{id}-figure")
    )